## 경로 설정

In [53]:
BASE_PATH = '/mnt/disk2/dl_data/open_sightseeing/'
ORG_TRAIN_CSV = 'train.csv'
NEW_TRAIN_CSV = 'train_except_mislabeled.csv'
KFOLDS_TRAIN_CSV = 'new_train_folds_5.csv'

### 잘못 레이블링된 데이터 제거

In [47]:
import pandas as pd
from sklearn import preprocessing
from glob import glob
import os

original_csv = os.path.join(BASE_PATH, ORG_TRAIN_CSV)
all_df = pd.read_csv(original_csv)
all_df

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [48]:
len(all_df)

16986

In [49]:
# 룰 기반
mislabeled_data_list = []
for i, row in all_df.iterrows():
    data_id = i
    img_path = row['img_path']
    img_filename = img_path.split("/")[-1]
    overview = row['overview']
    cat3 = row['cat3']
    #cat3_num = row['cat3_num']
    
    #if ('양식전문점이다' in overview or '정통 이탈리안' in overview) and '양식' not in cat3:
    if '양식전문점이다' in overview and '양식' not in cat3:
        mislabeled_data_list.append(data_id)
        print('양식 > ', img_filename, ' < ', cat3)
        print('         ', overview)
        
    #if ('일식전문점이다' in overview or '참치회,' in overview) and '일식' not in cat3:
    if '일식전문점이다' in overview and '일식' not in cat3:
        mislabeled_data_list.append(data_id)
        print('일식 > ', img_filename, ' < ', cat3)
        print('         ', overview)
        
    if '중식전문점이다' in overview and '중식' not in cat3:
        mislabeled_data_list.append(data_id)
        print('중식 > ', img_filename, ' < ', cat3)
        print('         ', overview)
        
    if '한식전문점이다' in overview and '한식' not in cat3:
        mislabeled_data_list.append(data_id)
        print('한식 > ', img_filename, ' < ', cat3)
        print('         ', overview)

print('')
print(mislabeled_data_list)
print('mislabeled_data_list count: ', len(mislabeled_data_list))

중식 >  TRAIN_01258.jpg  <  한식
          특이하고 다양한 중식요리를 맛볼 수 있는 곳이다. 대표메뉴는 물만두다. 서울특별시 영등포구에 있는 중식전문점이다.
한식 >  TRAIN_02764.jpg  <  바/까페
          저렴한 가격으로 독특한 찐빵을 맛볼 수 있는 곳이다. 대표메뉴는 찐빵이다. 경상북도 안동시에 있는 한식전문점이다.
한식 >  TRAIN_03803.jpg  <  일식
          신선한 회를 저렴한 가격으로 맛 볼수 있다. 대표메뉴는 모둠회다. 서울특별시 마포구에 있는 한식전문점이다.
한식 >  TRAIN_05186.jpg  <  서양식
          한국 미식 프로그램에 자주 소개된 매장이다. 대표메뉴는 꽃게찜이다. 서울특별시 성동구에 있는 한식전문점이다.
한식 >  TRAIN_05323.jpg  <  중식
          착한가격에 다양한 중식을 먹을 수 있는 곳이다. 서울특별시 종로구에 있는 한식전문점이다. 대표메뉴는 탕수육이다.
한식 >  TRAIN_05562.jpg  <  바/까페
          직접 로스팅한 커피 전문점이다. 서울특별시 종로구에 있는 한식전문점이다. 대표메뉴는 에스프레소이다.
한식 >  TRAIN_06084.jpg  <  서양식
          미쉐린 가이드 2021에 선정된 퓨전요리 전문점이다. 대표메뉴는 정식이다. 서울특별시 강남구에 있는 한식전문점이다.
중식 >  TRAIN_06952.jpg  <  한식
          다양한 중국식 요리를 맛볼 수 있는 곳이다. 대표메뉴는 탕수육이다. 서울특별시 영등포구에 있는 중식전문점이다.
양식 >  TRAIN_07726.jpg  <  바/까페
          다양한 양식 메뉴를 맛볼 수 있는 곳이다. 대표메뉴는 빠네파스타다. 서울특별시 중구에 있는 양식전문점이다.
한식 >  TRAIN_09024.jpg  <  중식
          을지로 직장인들에게 인기있는 매장이다. 서울특별시 중구에 있는 한식전문점이다. 대표메

In [50]:
# dropped mislabeled data
new_df = all_df.copy()
new_df.drop(mislabeled_data_list, inplace = True)

new_csv = os.path.join(BASE_PATH, NEW_TRAIN_CSV)
new_df.to_csv(new_csv, index=False)

print('done')

done


In [51]:
# 길이 체크
print(len(new_df))

16965


### Make Fold

In [52]:
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import pandas as pd

# os.environ["CURL_CA_BUNDLE"]=""

df = pd.read_csv(os.path.join(BASE_PATH, NEW_TRAIN_CSV))

In [54]:
# 레이블 인코딩
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df['cat1'].values)
df['cat1'] = le.transform(df['cat1'].values)
print('cat1 class count: ', len(le.classes_))

le = preprocessing.LabelEncoder()
le.fit(df['cat2'].values)
df['cat2'] = le.transform(df['cat2'].values)
print('cat2 class count: ', len(le.classes_))

le = preprocessing.LabelEncoder()
le.fit(df['cat3'].values)
df['cat3'] = le.transform(df['cat3'].values)
print('cat3 class count: ', len(le.classes_))

cat1 class count:  6
cat2 class count:  18
cat3 class count:  128


In [55]:
le.classes_

array(['5일장', 'ATV', 'MTB', '강', '게스트하우스', '계곡', '고궁', '고택', '골프', '공연장',
       '공예,공방', '공원', '관광단지', '국립공원', '군립공원', '기념관', '기념탑/기념비/전망대',
       '기암괴석', '기타', '기타행사', '농.산.어촌 체험', '다리/대교', '대중콘서트', '대형서점',
       '도립공원', '도서관', '동굴', '동상', '등대', '래프팅', '면세점', '모텔', '문', '문화관광축제',
       '문화원', '문화전수시설', '뮤지컬', '미술관/화랑', '민물낚시', '민박', '민속마을', '바/까페',
       '바다낚시', '박람회', '박물관', '발전소', '백화점', '번지점프', '복합 레포츠', '분수', '빙벽등반',
       '사격장', '사찰', '산', '상설시장', '생가', '서비스드레지던스', '서양식', '섬', '성',
       '수련시설', '수목원', '수상레포츠', '수영', '스노쿨링/스킨스쿠버다이빙', '스카이다이빙', '스케이트',
       '스키(보드) 렌탈샵', '스키/스노보드', '승마', '식음료', '썰매장', '안보관광', '야영장,오토캠핑장',
       '약수터', '연극', '영화관', '온천/욕장/스파', '외국문화원', '요트', '윈드서핑/제트스키',
       '유람선/잠수함관광', '유명건물', '유스호스텔', '유원지', '유적지/사적지', '이색거리', '이색찜질방',
       '이색체험', '인라인(실내 인라인 포함)', '일반축제', '일식', '자동차경주', '자연생태관광지',
       '자연휴양림', '자전거하이킹', '전문상가', '전시관', '전통공연', '종교성지', '중식', '채식전문점',
       '카약/카누', '카지노', '카트', '컨벤션', '컨벤션센터', '콘도미니엄', '클래식음악회', '클럽',
       '

In [56]:
df.head()

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,5,13,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,0,11,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,3,12,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,3,12,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,3,12,118


In [57]:
# k-fold 작업
#SPLIT_COUNT = 5
#SPLIT_RANDOM_STATE = 286
SPLIT_COUNT = 4
SPLIT_RANDOM_STATE = 286

folds = StratifiedKFold(n_splits=SPLIT_COUNT, random_state=SPLIT_RANDOM_STATE, shuffle=True)
df['kfold'] = -1
for i in range(SPLIT_COUNT):
    df_idx, valid_idx = list(folds.split(df.values, df['cat3']))[i]
    valid = df.iloc[valid_idx]

    df.loc[df[df.id.isin(valid.id) == True].index.to_list(), 'kfold'] = i

/home/ubuntu/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/ubuntu/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/ubuntu/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/ubuntu/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/ubuntu/venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [25]:
df.head()

,id,img_path,overview,cat1,cat2,cat3,kfold
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,5,13,120,0
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,0,11,8,0
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,3,12,118,1
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,3,12,118,3
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,3,12,118,2


In [60]:
df.to_csv(os.path.join(BASE_PATH, KFOLDS_TRAIN_CSV),index=False)
print('done')

done


### cat3 클래스별 데이터 개수 구하기

In [18]:
import pandas as pd

new_df = pd.read_csv(os.path.join(BASE_PATH, KFOLDS_TRAIN_CSV))

In [19]:
new_df.head()

,Unnamed: 0,id,img_path,overview,cat1,cat2,cat3,kfold
0,0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,5,13,120,0
1,1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,0,11,8,0
2,2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,3,12,118,1
3,3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,3,12,118,3
4,4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,3,12,118,2


In [20]:
classes = ['5일장', 'ATV', 'MTB', '강', '게스트하우스', '계곡', '고궁', '고택', '골프', '공연장',
       '공예,공방', '공원', '관광단지', '국립공원', '군립공원', '기념관', '기념탑/기념비/전망대',
       '기암괴석', '기타', '기타행사', '농.산.어촌 체험', '다리/대교', '대중콘서트', '대형서점',
       '도립공원', '도서관', '동굴', '동상', '등대', '래프팅', '면세점', '모텔', '문', '문화관광축제',
       '문화원', '문화전수시설', '뮤지컬', '미술관/화랑', '민물낚시', '민박', '민속마을', '바/까페',
       '바다낚시', '박람회', '박물관', '발전소', '백화점', '번지점프', '복합 레포츠', '분수', '빙벽등반',
       '사격장', '사찰', '산', '상설시장', '생가', '서비스드레지던스', '서양식', '섬', '성',
       '수련시설', '수목원', '수상레포츠', '수영', '스노쿨링/스킨스쿠버다이빙', '스카이다이빙', '스케이트',
       '스키(보드) 렌탈샵', '스키/스노보드', '승마', '식음료', '썰매장', '안보관광', '야영장,오토캠핑장',
       '약수터', '연극', '영화관', '온천/욕장/스파', '외국문화원', '요트', '윈드서핑/제트스키',
       '유람선/잠수함관광', '유명건물', '유스호스텔', '유원지', '유적지/사적지', '이색거리', '이색찜질방',
       '이색체험', '인라인(실내 인라인 포함)', '일반축제', '일식', '자동차경주', '자연생태관광지',
       '자연휴양림', '자전거하이킹', '전문상가', '전시관', '전통공연', '종교성지', '중식', '채식전문점',
       '카약/카누', '카지노', '카트', '컨벤션', '컨벤션센터', '콘도미니엄', '클래식음악회', '클럽',
       '터널', '테마공원', '트래킹', '특산물판매점', '패밀리레스토랑', '펜션', '폭포', '학교', '한식',
       '한옥스테이', '항구/포구', '해수욕장', '해안절경', '헬스투어', '헹글라이딩/패러글라이딩', '호수',
       '홈스테이', '희귀동.식물']

for i, classname in enumerate(classes):
    #print(i, ' ', classname)
    print(classname)

5일장
ATV
MTB
강
게스트하우스
계곡
고궁
고택
골프
공연장
공예,공방
공원
관광단지
국립공원
군립공원
기념관
기념탑/기념비/전망대
기암괴석
기타
기타행사
농.산.어촌 체험
다리/대교
대중콘서트
대형서점
도립공원
도서관
동굴
동상
등대
래프팅
면세점
모텔
문
문화관광축제
문화원
문화전수시설
뮤지컬
미술관/화랑
민물낚시
민박
민속마을
바/까페
바다낚시
박람회
박물관
발전소
백화점
번지점프
복합 레포츠
분수
빙벽등반
사격장
사찰
산
상설시장
생가
서비스드레지던스
서양식
섬
성
수련시설
수목원
수상레포츠
수영
스노쿨링/스킨스쿠버다이빙
스카이다이빙
스케이트
스키(보드) 렌탈샵
스키/스노보드
승마
식음료
썰매장
안보관광
야영장,오토캠핑장
약수터
연극
영화관
온천/욕장/스파
외국문화원
요트
윈드서핑/제트스키
유람선/잠수함관광
유명건물
유스호스텔
유원지
유적지/사적지
이색거리
이색찜질방
이색체험
인라인(실내 인라인 포함)
일반축제
일식
자동차경주
자연생태관광지
자연휴양림
자전거하이킹
전문상가
전시관
전통공연
종교성지
중식
채식전문점
카약/카누
카지노
카트
컨벤션
컨벤션센터
콘도미니엄
클래식음악회
클럽
터널
테마공원
트래킹
특산물판매점
패밀리레스토랑
펜션
폭포
학교
한식
한옥스테이
항구/포구
해수욕장
해안절경
헬스투어
헹글라이딩/패러글라이딩
호수
홈스테이
희귀동.식물


In [21]:
new_df.count()

Unnamed: 0    16965
id            16965
img_path      16965
overview      16965
cat1          16965
cat2          16965
cat3          16965
kfold         16965
dtype: int64

In [22]:
pd.set_option('display.max_rows', 200)
count_df = new_df['cat3'].value_counts()
count_df = count_df.sort_index(0)
count_dict = count_df.to_dict()
count_dict

/tmp/ipykernel_36589/1664361696.py:3: FutureWarning: In a future version of pandas all arguments of Series.sort_index will be keyword-only
  count_df = count_df.sort_index(0)


{0: 165,
 1: 4,
 2: 2,
 3: 104,
 4: 83,
 5: 160,
 6: 36,
 7: 92,
 8: 179,
 9: 120,
 10: 41,
 11: 271,
 12: 126,
 13: 21,
 14: 11,
 15: 64,
 16: 148,
 17: 53,
 18: 79,
 19: 71,
 20: 332,
 21: 41,
 22: 2,
 23: 17,
 24: 15,
 25: 92,
 26: 20,
 27: 18,
 28: 24,
 29: 18,
 30: 6,
 31: 340,
 32: 23,
 33: 20,
 34: 113,
 35: 26,
 36: 2,
 37: 204,
 38: 119,
 39: 39,
 40: 76,
 41: 781,
 42: 57,
 43: 29,
 44: 310,
 45: 4,
 46: 4,
 47: 4,
 48: 24,
 49: 8,
 50: 4,
 51: 9,
 52: 495,
 53: 239,
 54: 270,
 55: 43,
 56: 12,
 57: 223,
 58: 111,
 59: 85,
 60: 95,
 61: 95,
 62: 11,
 63: 27,
 64: 7,
 65: 2,
 66: 12,
 67: 60,
 68: 6,
 69: 21,
 70: 10,
 71: 27,
 72: 34,
 73: 1558,
 74: 17,
 75: 5,
 76: 4,
 77: 53,
 78: 7,
 79: 12,
 80: 27,
 81: 30,
 82: 105,
 83: 20,
 84: 211,
 85: 622,
 86: 467,
 87: 11,
 88: 6,
 89: 2,
 90: 549,
 91: 149,
 92: 35,
 93: 207,
 94: 118,
 95: 13,
 96: 173,
 97: 243,
 98: 6,
 99: 65,
 100: 207,
 101: 36,
 102: 10,
 103: 4,
 104: 7,
 105: 22,
 106: 11,
 107: 34,
 108: 2,
 109: 5,
 

In [23]:
for k, v in count_dict.items():
    print(v)

165
4
2
104
83
160
36
92
179
120
41
271
126
21
11
64
148
53
79
71
332
41
2
17
15
92
20
18
24
18
6
340
23
20
113
26
2
204
119
39
76
781
57
29
310
4
4
4
24
8
4
9
495
239
270
43
12
223
111
85
95
95
11
27
7
2
12
60
6
21
10
27
34
1558
17
5
4
53
7
12
27
30
105
20
211
622
467
11
6
2
549
149
35
207
118
13
173
243
6
65
207
36
10
4
7
22
11
34
2
5
8
172
246
37
60
445
40
9
3430
404
134
207
25
15
9
30
57
88


In [25]:
# 30% 를 validation 으로. 만약 30% 가 0 이면 1 로 한다.
data_count_dict = {}
val_count_dict = {}
for k, v in count_dict.items():
    # print(k, ' ', v)
    val_count = int(v * 0.3)
    if val_count == 0:
        val_count = 1
    #
    data_count_dict[k] = v
    val_count_dict[k] = val_count
    
val_count_dict

{0: 49,
 1: 1,
 2: 1,
 3: 31,
 4: 24,
 5: 48,
 6: 10,
 7: 27,
 8: 53,
 9: 36,
 10: 12,
 11: 81,
 12: 37,
 13: 6,
 14: 3,
 15: 19,
 16: 44,
 17: 15,
 18: 23,
 19: 21,
 20: 99,
 21: 12,
 22: 1,
 23: 5,
 24: 4,
 25: 27,
 26: 6,
 27: 5,
 28: 7,
 29: 5,
 30: 1,
 31: 102,
 32: 6,
 33: 6,
 34: 33,
 35: 7,
 36: 1,
 37: 61,
 38: 35,
 39: 11,
 40: 22,
 41: 234,
 42: 17,
 43: 8,
 44: 93,
 45: 1,
 46: 1,
 47: 1,
 48: 7,
 49: 2,
 50: 1,
 51: 2,
 52: 148,
 53: 71,
 54: 81,
 55: 12,
 56: 3,
 57: 66,
 58: 33,
 59: 25,
 60: 28,
 61: 28,
 62: 3,
 63: 8,
 64: 2,
 65: 1,
 66: 3,
 67: 18,
 68: 1,
 69: 6,
 70: 3,
 71: 8,
 72: 10,
 73: 467,
 74: 5,
 75: 1,
 76: 1,
 77: 15,
 78: 2,
 79: 3,
 80: 8,
 81: 9,
 82: 31,
 83: 6,
 84: 63,
 85: 186,
 86: 140,
 87: 3,
 88: 1,
 89: 1,
 90: 164,
 91: 44,
 92: 10,
 93: 62,
 94: 35,
 95: 3,
 96: 51,
 97: 72,
 98: 1,
 99: 19,
 100: 62,
 101: 10,
 102: 3,
 103: 1,
 104: 2,
 105: 6,
 106: 3,
 107: 10,
 108: 1,
 109: 1,
 110: 2,
 111: 51,
 112: 73,
 113: 11,
 114: 18,
 115: 13

### imbalanced data 에 대한 weight 구하기

In [26]:
import torch
data_count_list = list(data_count_dict.values())
data_count_list

[165,
 4,
 2,
 104,
 83,
 160,
 36,
 92,
 179,
 120,
 41,
 271,
 126,
 21,
 11,
 64,
 148,
 53,
 79,
 71,
 332,
 41,
 2,
 17,
 15,
 92,
 20,
 18,
 24,
 18,
 6,
 340,
 23,
 20,
 113,
 26,
 2,
 204,
 119,
 39,
 76,
 781,
 57,
 29,
 310,
 4,
 4,
 4,
 24,
 8,
 4,
 9,
 495,
 239,
 270,
 43,
 12,
 223,
 111,
 85,
 95,
 95,
 11,
 27,
 7,
 2,
 12,
 60,
 6,
 21,
 10,
 27,
 34,
 1558,
 17,
 5,
 4,
 53,
 7,
 12,
 27,
 30,
 105,
 20,
 211,
 622,
 467,
 11,
 6,
 2,
 549,
 149,
 35,
 207,
 118,
 13,
 173,
 243,
 6,
 65,
 207,
 36,
 10,
 4,
 7,
 22,
 11,
 34,
 2,
 5,
 8,
 172,
 246,
 37,
 60,
 445,
 40,
 9,
 3430,
 404,
 134,
 207,
 25,
 15,
 9,
 30,
 57,
 88]

In [28]:
weights = [1/x for x in data_count_list] #[ 1 / number of instances for each class]
class_weights = torch.FloatTensor(weights).cuda()
class_weights

tensor([6.0606e-03, 2.5000e-01, 5.0000e-01, 9.6154e-03, 1.2048e-02, 6.2500e-03,
        2.7778e-02, 1.0870e-02, 5.5866e-03, 8.3333e-03, 2.4390e-02, 3.6900e-03,
        7.9365e-03, 4.7619e-02, 9.0909e-02, 1.5625e-02, 6.7568e-03, 1.8868e-02,
        1.2658e-02, 1.4085e-02, 3.0120e-03, 2.4390e-02, 5.0000e-01, 5.8824e-02,
        6.6667e-02, 1.0870e-02, 5.0000e-02, 5.5556e-02, 4.1667e-02, 5.5556e-02,
        1.6667e-01, 2.9412e-03, 4.3478e-02, 5.0000e-02, 8.8496e-03, 3.8462e-02,
        5.0000e-01, 4.9020e-03, 8.4034e-03, 2.5641e-02, 1.3158e-02, 1.2804e-03,
        1.7544e-02, 3.4483e-02, 3.2258e-03, 2.5000e-01, 2.5000e-01, 2.5000e-01,
        4.1667e-02, 1.2500e-01, 2.5000e-01, 1.1111e-01, 2.0202e-03, 4.1841e-03,
        3.7037e-03, 2.3256e-02, 8.3333e-02, 4.4843e-03, 9.0090e-03, 1.1765e-02,
        1.0526e-02, 1.0526e-02, 9.0909e-02, 3.7037e-02, 1.4286e-01, 5.0000e-01,
        8.3333e-02, 1.6667e-02, 1.6667e-01, 4.7619e-02, 1.0000e-01, 3.7037e-02,
        2.9412e-02, 6.4185e-04, 5.8824e-

### sklearn 을 이용해서 imbalanced data 에 대한 weight 구하기

In [34]:
from sklearn.utils import class_weight

y_train = df["cat3"]
class_weights = class_weight.compute_class_weight(class_weight=data_count_dict,
                                                 classes=np.unique(y_train),
                                                 y=y_train)
class_weights

array([1.650e+02, 4.000e+00, 2.000e+00, 1.040e+02, 8.300e+01, 1.600e+02,
       3.600e+01, 9.200e+01, 1.790e+02, 1.200e+02, 4.100e+01, 2.710e+02,
       1.260e+02, 2.100e+01, 1.100e+01, 6.400e+01, 1.480e+02, 5.300e+01,
       7.900e+01, 7.100e+01, 3.320e+02, 4.100e+01, 2.000e+00, 1.700e+01,
       1.500e+01, 9.200e+01, 2.000e+01, 1.800e+01, 2.400e+01, 1.800e+01,
       6.000e+00, 3.400e+02, 2.300e+01, 2.000e+01, 1.130e+02, 2.600e+01,
       2.000e+00, 2.040e+02, 1.190e+02, 3.900e+01, 7.600e+01, 7.810e+02,
       5.700e+01, 2.900e+01, 3.100e+02, 4.000e+00, 4.000e+00, 4.000e+00,
       2.400e+01, 8.000e+00, 4.000e+00, 9.000e+00, 4.950e+02, 2.390e+02,
       2.700e+02, 4.300e+01, 1.200e+01, 2.230e+02, 1.110e+02, 8.500e+01,
       9.500e+01, 9.500e+01, 1.100e+01, 2.700e+01, 7.000e+00, 2.000e+00,
       1.200e+01, 6.000e+01, 6.000e+00, 2.100e+01, 1.000e+01, 2.700e+01,
       3.400e+01, 1.558e+03, 1.700e+01, 5.000e+00, 4.000e+00, 5.300e+01,
       7.000e+00, 1.200e+01, 2.700e+01, 3.000e+01, 

### Splitter (클래스별 30% 를 validation 으로 분리)

In [38]:
from glob import glob
import os
import shutil
import random
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import preprocessing

BASE_PATH = '/home/jovyan/dl_data/open_sightseeing/'
ORG_CSV = 'new_train_folds_5.csv'
NEW_CSV = 'new_trian_with_flag_set3.csv'

org_csv = os.path.join(BASE_PATH, ORG_CSV)
org_df = pd.read_csv(org_csv)
org_df = shuffle(org_df, random_state=2022)

remain_val_count_dict = val_count_dict.copy()
new_df = org_df.copy()
new_df["is_valid"] = -1

for i, row in tqdm(org_df.iterrows(), total=org_df.shape[0]):
    cat3 = int(row['cat3'])
    
    # remain_val_count_dict 에서 cat3_num 에 해당하는 개수가 0 보다 크면 validation 으로, 아니면 train 으로 분리
    
    remain_valid_count = remain_val_count_dict[cat3]
    if remain_valid_count == 0:
        # train
        new_df.loc[i, 'is_valid'] = 0
    else:
        # valid
        new_df.loc[i, 'is_valid'] = 1
        #        
        remain_valid_count -= 1
        remain_val_count_dict[cat3] = remain_valid_count

new_df.to_csv(os.path.join(BASE_PATH, NEW_CSV), index=False)

print('done')

100%|██████████| 16965/16965 [00:05<00:00, 3005.64it/s]


done


### .csv 에서 value 컬럼 제거

In [40]:
import pandas as pd

BASE_PATH = '/home/jovyan/dl_data/open_sightseeing/'
WITH_VALUE_CSV = 'my_best_b2_with_value.csv'
WITHOUT_VALUE_CSV = 'my_best_b2.csv'

temp_df = pd.read_csv(os.path.join(BASE_PATH, WITH_VALUE_CSV))
temp_df.drop('cat3_value', inplace=True, axis=1)
temp_df.to_csv(os.path.join(BASE_PATH, WITHOUT_VALUE_CSV), index=False)

print('done')

done


### 이미지 해상도 구하기

In [42]:
from glob import glob
import os
from tqdm import tqdm
import cv2

base_path = '/home/jovyan/dl_data/open_sightseeing/image/train/'

file_list = glob(base_path + '*.jpg')
file_count = len(file_list)
print(file_count)

min_width = 99999
min_height = 99999
max_width = 0
max_height = 0

for file in tqdm(file_list):
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    h, w, c = img.shape
    
    if w < min_width:
        min_width = w
    if h < min_height:
        min_height = h
    if w > max_width:
        max_width = w
    if h > max_height:
        max_height = h
    
print('min : ', min_width, ' x ', min_height)
print('max : ', max_width, ' x ', max_height)

print('done')

16986


100%|██████████| 16986/16986 [02:20<00:00, 121.01it/s]

min :  160  x  96
max :  6192  x  4128
done
